In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256
Corr>0.95: 139 Now:
Corr Target <0.1: 4206


## Test Improvement

In [27]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(np.sqrt(np.mean((y_pred - y_true)**2))) 

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'l2', 'root_mean_squared_error'},
    'num_leaves': 100,
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

print('Start predicting...')
# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', root_mean_squared_error(y_test, y_pred))

Start training...
[1]	valid_0's rmse: 1.69953	valid_0's l2: 2.8884
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's rmse: 1.66258	valid_0's l2: 2.76419
[3]	valid_0's rmse: 1.63028	valid_0's l2: 2.65783
[4]	valid_0's rmse: 1.60201	valid_0's l2: 2.56644
[5]	valid_0's rmse: 1.57966	valid_0's l2: 2.49533
[6]	valid_0's rmse: 1.55799	valid_0's l2: 2.42734
[7]	valid_0's rmse: 1.53682	valid_0's l2: 2.36183
[8]	valid_0's rmse: 1.52309	valid_0's l2: 2.3198
[9]	valid_0's rmse: 1.51052	valid_0's l2: 2.28168
[10]	valid_0's rmse: 1.499	valid_0's l2: 2.24699
[11]	valid_0's rmse: 1.48625	valid_0's l2: 2.20894
[12]	valid_0's rmse: 1.47755	valid_0's l2: 2.18315
[13]	valid_0's rmse: 1.47009	valid_0's l2: 2.16116
[14]	valid_0's rmse: 1.46215	valid_0's l2: 2.13788
[15]	valid_0's rmse: 1.45547	valid_0's l2: 2.11838
[16]	valid_0's rmse: 1.44763	valid_0's l2: 2.09563
[17]	valid_0's rmse: 1.44375	valid_0's l2: 2.08442
[18]	valid_0's rmse: 1.43871	valid_0's l2: 2.0699
[19]	valid_0's rm

In [28]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'l2', 'root_mean_squared_error'},
    'num_leaves': 100,
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(gbm.predict(x_train, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(gbm.predict(x_test, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

[1]	valid_0's rmse: 1.69953	valid_0's l2: 2.8884
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's rmse: 1.66258	valid_0's l2: 2.76419
[3]	valid_0's rmse: 1.63028	valid_0's l2: 2.65783
[4]	valid_0's rmse: 1.60201	valid_0's l2: 2.56644
[5]	valid_0's rmse: 1.57966	valid_0's l2: 2.49533
[6]	valid_0's rmse: 1.55799	valid_0's l2: 2.42734
[7]	valid_0's rmse: 1.53682	valid_0's l2: 2.36183
[8]	valid_0's rmse: 1.52309	valid_0's l2: 2.3198
[9]	valid_0's rmse: 1.51052	valid_0's l2: 2.28168
[10]	valid_0's rmse: 1.499	valid_0's l2: 2.24699
[11]	valid_0's rmse: 1.48625	valid_0's l2: 2.20894
[12]	valid_0's rmse: 1.47755	valid_0's l2: 2.18315
[13]	valid_0's rmse: 1.47009	valid_0's l2: 2.16116
[14]	valid_0's rmse: 1.46215	valid_0's l2: 2.13788
[15]	valid_0's rmse: 1.45547	valid_0's l2: 2.11838
[16]	valid_0's rmse: 1.44763	valid_0's l2: 2.09563
[17]	valid_0's rmse: 1.44375	valid_0's l2: 2.08442
[18]	valid_0's rmse: 1.43871	valid_0's l2: 2.0699
[19]	valid_0's rmse: 1.43649	valid_

Epoch 67/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.0518 - val_loss: 1.2486
Epoch 68/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.0296 - val_loss: 1.2319
Epoch 69/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.0195 - val_loss: 1.2480
Epoch 70/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.0228 - val_loss: 1.2399
Epoch 71/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.0377 - val_loss: 1.2356
Epoch 72/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.0192 - val_loss: 1.2404
Epoch 73/1000
3567/3567 [==============================] - 0s 18us/step - loss: 1.0361 - val_loss: 1.2318
Epoch 74/1000
3567/3567 [==============================] - 0s 17us/step - loss: 1.0106 - val_loss: 1.2343
Epoch 75/1000
3567/3567 [==============================] - 0s 17us/step - loss: 1.0047 - val_loss: 1.2401
Epoch 76/1000
3567/3567 [=====================

## NN+ElNet - Actual

In [31]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)))

lgb_train = lgb.Dataset(x_train, y_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'l2', 'root_mean_squared_error'},
    'num_leaves': 100,
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20)

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(gbm.predict(x_train, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(gbm.predict(x_test, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s 89us/step - loss: 13.5217
Epoch 2/1000
4459/4459 [==============================] - 0s 16us/step - loss: 9.6060
Epoch 3/1000
4459/4459 [==============================] - 0s 15us/step - loss: 7.9935
Epoch 4/1000
4459/4459 [==============================] - 0s 15us/step - loss: 6.6946
Epoch 5/1000
4459/4459 [==============================] - 0s 15us/step - loss: 5.6017
Epoch 6/1000
4459/4459 [==============================] - ETA: 0s - loss: 4.936 - 0s 16us/step - loss: 4.9369
Epoch 7/1000
4459/4459 [==============================] - 0s 15us/step - loss: 4.2768
Epoch 8/1000
4459/4459 [==============================] - 0s 15us/step - loss: 3.9596
Epoch 9/1000
4459/4459 [==============================] - 0s 15us/step - loss: 3.3072
Epoch 10/1000
4459/4459 [==============================] - 0s 15us/step - loss: 2.8932
Epoch 11/1000
4459/4459 [==============================] - 0s 15us/step - loss: 2.5767
Epoch 12/1000
4459/4459 [==

4459/4459 [==============================] - 0s 15us/step - loss: 1.1151
Epoch 95/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1320
Epoch 96/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1272
Epoch 97/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1276
Epoch 98/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1116
Epoch 99/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1219
Epoch 100/1000
4459/4459 [==============================] - 0s 16us/step - loss: 1.1129
Epoch 101/1000
4459/4459 [==============================] - 0s 16us/step - loss: 1.1166
Epoch 102/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0985
Epoch 103/1000
4459/4459 [==============================] - 0s 16us/step - loss: 1.1022
Epoch 104/1000
4459/4459 [==============================] - 0s 16us/step - loss: 1.1028
Epoch 105/1000
4459/4459 [==========================

4459/4459 [==============================] - 0s 16us/step - loss: 1.0654
Epoch 188/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0576
Epoch 189/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0570
Epoch 190/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0465
Epoch 191/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0585
Epoch 192/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0655
Epoch 193/1000
4459/4459 [==============================] - 0s 16us/step - loss: 1.0656
Epoch 194/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0498
Epoch 195/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0513
Epoch 196/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0534
Epoch 197/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.0667
Epoch 198/1000
4459/4459 [=====================

In [32]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_lightgbm.csv',index=False)

49342/49342 [==============================] - 1s 18us/step
          ID       target
0  000137c73  1363495.125
1  00021489f  2345308.000
2  0004d7953  4020863.250
3  00056a333  2726191.750
4  00056d8eb  2345308.000
